<a href="https://colab.research.google.com/github/karoldem/dh/blob/master/dh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy

class dh:
  def __init__(self,n):
    self.table = []
    for i in range (n):
      self.table.append({'a': sympy.symbols(r'a_' + str(i+1), real = True),
                  'al':sympy.symbols(r'\alpha_' + str(i+1), real = True),
                  'd': sympy.symbols(r'd_' +  str(i+1), real = True),
                  't': sympy.symbols(r'\theta_' +  str(i+1), real = True),
                  'x':'t'})

  def dhm(self,i):
    """
    Creates transform matrix of a joint. 
    """
    return (sympy.Matrix([  [sympy.cos(i['t']), -sympy.sin(i['t'])*sympy.cos(i['al']), sympy.sin(i['t'])*sympy.sin(i['al']), i['a']*sympy.cos(i['t'])],
                            [sympy.sin(i['t']), sympy.cos(i['t'])*sympy.cos(i['al']), -sympy.cos(i['t'])*sympy.sin(i['al']), i['a']*sympy.sin(i['t'])],
                            [0, sympy.sin(i['al']), sympy.cos(i['al']), i['d']],
                            [0,0,0,1]  ]))
    
  def makematrix (self):
    t = sympy.eye(4)
    for i in self.table:
      t = t*self.dhm(i)
    return t
  
  def rotX(self,a):
    return sympy.Matrix([  [1,0,0,0],
                           [0,sympy.cos(a), -sympy.sin(a),0],
                           [0,sympy.sin(a), sympy.cos(a),0],
                           [0,0,0,1]  ])
  
  def rotY(self,a):
    return sympy.Matrix([  [sympy.cos(a), 0, sympy.sin(a),0],
                           [0,1,0,0],
                           [-sympy.sin(a),0,sympy.cos(a),0],
                           [0,0,0,1]  ])
  def rotZ(self,a):
    return sympy.Matrix([  [sympy.cos(a), -sympy.sin(a),0,0],
                           [sympy.sin(a), sympy.cos(a),0,0],
                           [0,0,1,0],
                           [0,0,0,1]  ])

  def jacobian_matrix (self):
    correct_x = sympy.Matrix(1,0,[])
    for i in self.table:
      correct_x = sympy.Matrix([ i[ i['x'] ] ]).col_insert(0,correct_x)
      pass
    
    t = sympy.eye(4)
    for i in self.table:
      t = t*self.dhm(i)

    self.placement = t * sympy.Matrix([[0,0,0,1]]).T
    j = self.placement.T.jacobian( correct_x )
    j.row_del(3)

    for i in range(len( j )):
      j[i] = sympy.simplify( j[i] )
    
    return j
  
  def jacobian (self): return sympy.simplify( self.jacobian_matrix().det() )

  def singular_config(self): return list(map(lambda x: sympy.solve (self.jacobian(), x[x['x']] ), self.table))

  def print_jacobian_matrix (self):
    print (r'$$')
    print (r'J = ' + str(sympy.latex( self.jacobian_matrix() )))
    print (r"$$")
    print (r'')

    
  def print_transorm_matrices(self):
    for i in range (len( self.table )):
      t = self.dhm( self.table[i] )

      for j in range (16):
        t[j] = sympy.simplify(t[j])
      
      print (r'$$')
      print (r'^' + str(i) + r'T_' +str(i+1) +  r' = ')
      print(str(sympy.latex(t)))
      print (r'$$')

    print (r'$$')
    print (r'^0 T_'+ str(len(self.table)) + r' = ')
    print(str(sympy.latex( self.makematrix() )))
    print (r'$$')
  
  def forward_kinematics_3D(self):
    m = self.makematrix()*sympy.Matrix([[0],[0],[0],[1]])
    for i in range (4):
      m[i] = sympy.simplify(m[i])
    return m
    

In [ ]:
primitive = dh(2)

primitive.table[0]['d'] = 0
primitive.table[0]['al'] = 0

primitive.table[1]['d'] = 0
primitive.table[1]['al'] = 0

k = primitive.forward_kinematics_3D()

sympy.solve([k[0], k[1]], [primitive.table[0]['t'], primitive.table[1]['t']])

[(pi, -acos(-a_1/a_2) + 2*pi),
 (pi, acos(-a_1/a_2)),
 (-I*log((a_2 - sqrt(-a_1**2 + a_2**2))/a_1),
  -I*log(-a_1/(a_2 - sqrt(-a_1**2 + a_2**2)))),
 (-I*log((a_2 + sqrt(-a_1**2 + a_2**2))/a_1),
  -I*log(-a_1/(a_2 + sqrt(-a_1**2 + a_2**2))))]

In [ ]:
cartesian = dh(3)
cartesian.table[0]['a'] = 0
cartesian.table[1]['a'] = 0
cartesian.table[2]['a'] = 0

cartesian.table[0]['t'] = 0
cartesian.table[0]['al'] = -sympy.rad(90)
cartesian.table[0]['x'] = 'd'

cartesian.table[1]['t'] = sympy.rad(90)
cartesian.table[1]['al'] = sympy.rad(90)
cartesian.table[1]['x'] = 'd'

cartesian.table[2]['t'] = 0
cartesian.table[2]['al'] = 0
cartesian.table[2]['x'] = 'd'

cartesian.forward_kinematics_3D()

def reverse(pos):
  return [pos(2), pos(1), pos(0)]


Matrix([
[d_3],
[d_2],
[d_1],
[  1]])

Here we define our robot constructor created dh table in field `table` with special kind of variables (google sympy library for more). To define a robot, user must replace those variables with constants (or any other type - free country).
If you happen to contribute, let us agree, that this is what makes a robot waht it is and any other method must base on it (unlike in FreeCAD project if you know what I mean)

In [ ]:
antropomorphic = dh(3) #constructor creates dh table with special kind of variables (google sympy library for more)

#here we replace those variables with constatns (al means alpha, t means theta)
antropomorphic.table[0]['a'] = 0
antropomorphic.table[0]['al'] = sympy.rad(90)

antropomorphic.table[1]['d'] = 0
antropomorphic.table[1]['al'] = 0

antropomorphic.table[2]['d'] = 0
antropomorphic.table[2]['al'] = 0

antropomorphic.forward_kinematics_3D()

Matrix([
[(a_2*cos(\theta_2) + a_3*cos(\theta_2 + \theta_3))*cos(\theta_1)],
[(a_2*cos(\theta_2) + a_3*cos(\theta_2 + \theta_3))*sin(\theta_1)],
[          a_2*sin(\theta_2) + a_3*sin(\theta_2 + \theta_3) + d_1],
[                                                               1]])

In [ ]:
antropomorphic.print_jacobian_matrix()

$$
J = \left[\begin{matrix}- \left(a_{2} \cos{\left (\theta_2 \right )} + a_{3} \cos{\left (\theta_2 + \theta_3 \right )}\right) \sin{\left (\theta_1 \right )} & - \left(a_{2} \sin{\left (\theta_2 \right )} + a_{3} \sin{\left (\theta_2 + \theta_3 \right )}\right) \cos{\left (\theta_1 \right )} & - a_{3} \sin{\left (\theta_2 + \theta_3 \right )} \cos{\left (\theta_1 \right )}\\\left(a_{2} \cos{\left (\theta_2 \right )} + a_{3} \cos{\left (\theta_2 + \theta_3 \right )}\right) \cos{\left (\theta_1 \right )} & - \left(a_{2} \sin{\left (\theta_2 \right )} + a_{3} \sin{\left (\theta_2 + \theta_3 \right )}\right) \sin{\left (\theta_1 \right )} & - a_{3} \sin{\left (\theta_1 \right )} \sin{\left (\theta_2 + \theta_3 \right )}\\0 & a_{2} \cos{\left (\theta_2 \right )} + a_{3} \cos{\left (\theta_2 + \theta_3 \right )} & a_{3} \cos{\left (\theta_2 + \theta_3 \right )}\end{matrix}\right]
$$



method `dhm(n)` creates matrix of transformation between nth and (n-1)th coordinate system.

In [ ]:
antropomorphic.dhm(antropomorphic.table[2])

Matrix([
[cos(\theta_3), -sin(\theta_3), 0, a_3*cos(\theta_3)],
[sin(\theta_3),  cos(\theta_3), 0, a_3*sin(\theta_3)],
[            0,              0, 1,                 0],
[            0,              0, 0,                 1]])

method makematrix() returns transform matrix between zeroth system and TCP

In [ ]:
antropomorphic.makematrix()

Matrix([
[-sin(\theta_2)*sin(\theta_3)*cos(\theta_1) + cos(\theta_1)*cos(\theta_2)*cos(\theta_3), -sin(\theta_2)*cos(\theta_1)*cos(\theta_3) - sin(\theta_3)*cos(\theta_1)*cos(\theta_2),  sin(\theta_1), a_2*cos(\theta_1)*cos(\theta_2) - a_3*sin(\theta_2)*sin(\theta_3)*cos(\theta_1) + a_3*cos(\theta_1)*cos(\theta_2)*cos(\theta_3)],
[-sin(\theta_1)*sin(\theta_2)*sin(\theta_3) + sin(\theta_1)*cos(\theta_2)*cos(\theta_3), -sin(\theta_1)*sin(\theta_2)*cos(\theta_3) - sin(\theta_1)*sin(\theta_3)*cos(\theta_2), -cos(\theta_1), a_2*sin(\theta_1)*cos(\theta_2) - a_3*sin(\theta_1)*sin(\theta_2)*sin(\theta_3) + a_3*sin(\theta_1)*cos(\theta_2)*cos(\theta_3)],
[                             sin(\theta_2)*cos(\theta_3) + sin(\theta_3)*cos(\theta_2),                             -sin(\theta_2)*sin(\theta_3) + cos(\theta_2)*cos(\theta_3),              0,                                     a_2*sin(\theta_2) + a_3*sin(\theta_2)*cos(\theta_3) + a_3*sin(\theta_3)*cos(\theta_2) + d_1],
[                     

now we print all matrices

In [ ]:
antropomorphic.print_transorm_matrices()

$$
^0T_1 = 
\left[\begin{matrix}\cos{\left (\theta_1 \right )} & 0 & \sin{\left (\theta_1 \right )} & 0\\\sin{\left (\theta_1 \right )} & 0 & - \cos{\left (\theta_1 \right )} & 0\\0 & 1 & 0 & d_{1}\\0 & 0 & 0 & 1\end{matrix}\right]
$$
$$
^1T_2 = 
\left[\begin{matrix}\cos{\left (\theta_2 \right )} & - \sin{\left (\theta_2 \right )} & 0 & a_{2} \cos{\left (\theta_2 \right )}\\\sin{\left (\theta_2 \right )} & \cos{\left (\theta_2 \right )} & 0 & a_{2} \sin{\left (\theta_2 \right )}\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$
$$
^2T_3 = 
\left[\begin{matrix}\cos{\left (\theta_3 \right )} & - \sin{\left (\theta_3 \right )} & 0 & a_{3} \cos{\left (\theta_3 \right )}\\\sin{\left (\theta_3 \right )} & \cos{\left (\theta_3 \right )} & 0 & a_{3} \sin{\left (\theta_3 \right )}\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$
$$
^0 T_3 = 
\left[\begin{matrix}- \sin{\left (\theta_2 \right )} \sin{\left (\theta_3 \right )} \cos{\left (\theta_1 \right )} + \cos{\left (\theta_1 \right )} \c

In [ ]:
spheric = dh(3)
spheric.table[0]['d'] = 0
spheric.table[0]['a'] = 0
spheric.table[0]['al'] = sympy.rad(90)

spheric.table[1]['d'] = 0
spheric.table[1]['a'] = 0
spheric.table[1]['al'] = sympy.rad(-90)

spheric.table[2]['t'] = 0
spheric.table[2]['a'] = 0
spheric.table[2]['al'] = 0
spheric.table[2]['x'] = 'd'

In [ ]:
cylindrical = dh(3)
cylindrical.table[0]['d'] = 0
cylindrical.table[0]['a'] = 0
cylindrical.table[0]['al'] = 0

cylindrical.table[1]['t'] = 0
cylindrical.table[1]['a'] = 0
cylindrical.table[1]['al'] = sympy.rad(90)
cylindrical.table[1]['x'] = 'd'

cylindrical.table[2]['t'] = 0
cylindrical.table[2]['a'] = 0
cylindrical.table[2]['al'] = sympy.rad(-90)
cylindrical.table[2]['x'] = 'd'

In [ ]:
scara = dh(3)
scara.table[0]['al'] = 0

scara.table[1]['d'] = 0
scara.table[1]['al'] = sympy.rad(180)

scara.table[2]['t'] = 0
scara.table[2]['a'] = 0
scara.table[2]['al'] = 0
scara.table[2]['x'] = 'd'

In [ ]:
sympy.expand_trig(sympy.sin(sympy.symbols('x') + sympy.symbols('y')))

sin(x)*cos(y) + sin(y)*cos(x)

In [ ]:
import sympy

x,y = sympy.symbols('x,y', real = True)
f =     (x-1)**2  +  3*x*y  -  (y-2)**2
#x = sympy.solve(sympy.diff(f,x))[0][x]
#f =     (x-1)**2  +  3*x*y  -  (y-2)**2
#sympy.solve(sympy.diff(f,y))
sympy.solve(sympy.diff(f,x))

[{x: -3*y/2 + 1}]

In [ ]:
import sympy

x,y = sympy.symbols('x,y', real = True)
f =     (x-1)**2  +  3*x*y  -  (y-2)**2
sympy.solve(sympy.diff(f,x))
x = sympy.solve(sympy.diff(f,x))[0][x]
f =     (x-1)**2  +  3*x*y  -  (y-2)**2
sympy.solve(sympy.diff(f,y))


[14/13]

In [ ]:
-3.*(14/13)/2. + 1, 14/13

(-0.6153846153846154, 1.0769230769230769)

In [ ]:
import sympy

x,y = sympy.symbols('x,y', real = True)
f =     (x-1)**2  +  3*x*y  -  (y-2)**2

fxx = sympy.diff(sympy.diff(f,x),x)
fyy = sympy.diff(sympy.diff(f,y),y)
fxy = sympy.diff(sympy.diff(f,y),x)

w = sympy.Matrix([[fxx, fxy],[fxy, fyy]])

sympy.det(w)


fxx,fxy, fyy


(2, 3, 2)

In [ ]:
import sympy

x,y = sympy.symbols('x,y', real = True)
f =     (x-1)**2  +  3*x*y  -  (y-2)**2

fxx = sympy.diff(sympy.diff(f,x),x)
fyy = sympy.diff(sympy.diff(f,y),y)
fxy = sympy.diff(sympy.diff(f,y),x)

fxx, sympy.diff(f,x)

(2, 2*x + 3*y - 2)

In [ ]:
2*1.6 - 3*0.4 - 2

0.0